<a href="https://colab.research.google.com/github/tb-harris/neuroscience-2025/blob/main/9_Multiple_Regression_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import pandas as pd # pandas (DataFrames)
import seaborn as sns # graphs
import statsmodels.formula.api as smf # regression models
# installing and importing the custom diagnostics module
%pip install git+https://github.com/tb-harris/diagnostics.git
from diagnostics import LinearRegDiagnostic

  Cloning https://github.com/tb-harris/diagnostics.git to /tmp/pip-req-build-em6l32qc
  Running command git clone --filter=blob:none --quiet https://github.com/tb-harris/diagnostics.git /tmp/pip-req-build-em6l32qc
  Resolved https://github.com/tb-harris/diagnostics.git to commit 63e7ed899813d387d104118e3b35d435b398c0df
  Preparing metadata (setup.py) ... done
  Created wheel for diagnostics: filename=diagnostics-0.1-py3-none-any.whl size=4726 sha256=9b97ff6feaa2b64809a633b26f9b72a19e4dc8ac385e2df4bbd2586dfe6d31dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-qejfsni1/wheels/dc/6f/7f/5d7dbbdace3319b3bc68e42a4691a9f8e3605e00b9f8b8ce2b
Successfully built diagnostics


In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read in the data

In [ ]:
cell_metadata = pd.read_csv("/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/20200711_patchseq_metadata_mouse.csv", index_col="cell_specimen_id")

rna_data = pd.read_hdf("/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/gene_counts_ppm.hdf5.lz4")

morph_additional = pd.read_csv(
    "/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/morph_features_expanded.csv",
    index_col="cell_specimen_id"
).dropna(how="all")

morph_positional = pd.read_csv(
    "/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/morph_features_positional.csv",
    index_col="cell_specimen_id"
).dropna(how="all")

### Make DataFrames

* morph_full_recon has only morphology
* rna_data has only gene data
* gene_morph has morphology, gene, and metadata

These only include cells that have full reconstructions

In [ ]:
morph_full_recon = morph_additional[cell_metadata["neuron_reconstruction_type"] == "full"]
rna_data = rna_data[cell_metadata["neuron_reconstruction_type"] == "full"]
rna_data = rna_data.loc[morph_full_recon.index]

# drop zero genes
nonzero_genes = rna_data.sum() != 0
rna_data = rna_data.loc[:, nonzero_genes]

# dataframe with ALL info (rna, morph, metadata)
gene_morph = morph_full_recon.join(rna_data).join(cell_metadata)

/tmp/ipython-input-26-4073318482.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  morph_full_recon = morph_additional[cell_metadata["neuron_reconstruction_type"] == "full"]
/tmp/ipython-input-26-4073318482.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rna_data = rna_data[cell_metadata["neuron_reconstruction_type"] == "full"]
